In [ ]:
import numpy as np 
import time
from time import sleep
from IPython.display import HTML, display, clear_output
import sys

import requests
from bs4 import BeautifulSoup
import json

In [ ]:
def get_img(img_file):
    
    #Function to set the width of the image to a reasonable size.
    
    style_str = "width: 600px;"
    html_out = HTML(f"<img style = '{style_str}' src = {img_file}> </img>")
    display(html_out)

In [ ]:
# Files, questions and answers are all in list format, separated to three parts corresponding to parts of the question.
# In loops, all corresponding elements have the same index.

files = ["test1.png", "test2.png", "test3.png"]

questions = [
    ["The colour of the triangle is ...?", 
     "What is the shape on the right?",
     "The colour of the shape on the left is ...?"
     ],
    
    ["What shape was purple?", 
     "The shape on the top left is ...?",
     "Which shape was below the circle?", 
     "How many sides does the red shape have?", 
     "What colour was the triangle?"
    ],
    
    ["What was the shape between the star and the pentagon?", 
     "What was the shape above the sun, star, square, cross of heart?",
     "The colour of the triangle was ... ?", 
     "What was the colour of the smiley face?", 
     "The colour of the circle is ... ?"
     ]
]
          

answers = [
    ["blue", "square", "blue"],
    ["star", "circle", "triangle", "4", "blue"],
    ["square", "heart", "black", "yellow", "red"]
]


In [ ]:
score = 0

def test(img_file, q_list, a_list):

    global score

    print('You have 12 seconds to look and memorise the table below')
    get_img(img_file)
    time.sleep(12)
    clear_output(wait=False)

    #Clears image after designated time.

    time.sleep(1)

    for i in range(len(q_list)):

        print(f'Your current score is {score} points.')

        ans = input(q_list[i])
        if ans.lower() == a_list[i]:
            
            #prevents falsely incorrect input due to capitalisations.
            
            print("Well done!")
            score += 1+(i*0.5)
            
            #Questions get harder due to memory loss as time passes. 0.5 points are added for consecutive questions.
        
        else: 
            print(f'The correct answer is {a_list[i]}')

       
        time.sleep(1)
        clear_output(wait=False)
        #Clears the screen after each question to prevent overpiling.
        

In [ ]:
def run_test():

    global score
    
    for i in range(len(files)):
        test(files[i], questions[i], answers[i])
        #Goes through each "set" of grid, its questions and the answers.
        
    print(f'Your total score is {score} points.')

In [ ]:
data_consent_info = """DATA CONSENT INFORMATION:

Please read:

we wish to record your response data
to an anonymised public data repository. 
Your data will be used for educational teaching purposes
practising data analysis and visualisation.

Please type  "yes"   in the box below if you consent to the upload."""

print(data_consent_info)
consent = input("> ") 

clear_output(wait=False)

name = input("Please enter your username.") 

gender = input("Please enter your gender (male, female or other)")

print("How much sleep have you had last night?")
sleep = input("Enter your number of hours as a single number (e.g. 8).") 

print("How many days a week do you exercise for more than 30 minutes?")
exercise = input("Enter your answer as a single number.")

clear_output(wait=False)



data_dict = {
        "Name":name,
        "Score":score,
        "Sleep":sleep,
        "Exercise":exercise,
        "Gender":gender
    }

form_url = "https://docs.google.com/forms/d/e/1FAIpQLSewE5xIXJvVJDZbKCl0ykLtDezU5RmQFL3VRyptMi4jif09Tg/viewform"

def send_to_google_form(data_dict, form_url):

    form_id = form_url[34:90]
    view_form_url = f'https://docs.google.com/forms/d/e/{form_id}/viewform'
    post_form_url = f'https://docs.google.com/forms/d/e/{form_id}/formResponse'

    page = requests.get(view_form_url)
    content = BeautifulSoup(page.content, "html.parser").find('script', type='text/javascript')
    content = content.text[27:-1]
    result = json.loads(content)[1][1]
    form_dict = {}
    
    #Obtains json data from the URL provided. Allowing identification of each section to be filled in the Google Form
    
    loaded_all = True
    for item in result:
        if item[1] not in data_dict:
            print(f"Form item {item[1]} not found. Data not uploaded.")
            loaded_all = False
            return False
            #Ensures all the elements in data_dict are present in the form. 
        form_dict[f'entry.{item[4][0][0]}'] = data_dict[item[1]]
    
    post_result = requests.post(post_form_url, data=form_dict)
    return post_result.ok

run_test()

if consent.lower() == "yes":
    send_to_google_form(data_dict, form_url)
    print("Thank you for completing this quiz. Your results have been uploaded.")
else:
    print("We will not collect your data. Hope you have had fun.")